In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from torch.optim.optimizer import Optimizer
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
from transformers import pipeline
import warnings 
warnings.filterwarnings('ignore')
from pytorch_lightning import seed_everything
from torch.utils.data import DataLoader
import os
import gc
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForQuestionAnswering
import pandas as pd
from tqdm import tqdm_notebook as tqdm

gc.collect()

def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def set_seed(seed = int):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    seed_everything(seed)
    return random_state
random_state = set_seed(42)


model_path = '../models/Product_Data_SQuAD_model_product.pt'
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")
model.load_state_dict(torch.load(model_path))
model.eval()
nlp = pipeline('question-answering', model=model.to('cpu'), tokenizer=tokenizer)

def model_predict(nlp,df):
    table = pd.DataFrame()
    for i in tqdm(sorted(df.index.tolist())):
        sample = df.loc[[i]]
        string_X_train = sample['string_X_train'].values[0]
        QA_input = {
            'question': 'What is the product name?',
            'context': string_X_train
        }
        res = nlp(QA_input)
        predict = QA_input['context'][res['start']:res['end']]
        row = pd.DataFrame({'predict:':predict},index=[i])
        table = table.append(row)
    return table
gc.collect()

C:\Users\ricky\anaconda3\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
Global seed set to 42
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Som

0

In [2]:
a = ['apple','apple 11','apple 11 plus']
def substringSieve(string_list):
    out = []
    for s in string_list:
        if not any([s in r for r in string_list if s != r]):
            out.append(s)
    return out
print(a,substringSieve(a))


['apple', 'apple 11', 'apple 11 plus'] ['apple 11 plus']


# 寶典

In [3]:
df = pd.read_excel('../data/台塑企業_ 產品寶典20210303.xlsx',engine='openpyxl').iloc[:,:-1]
df2 = pd.read_excel('../data/寶典.v3.台塑網.20210901.xlsx',engine='openpyxl')
df2.columns = df.columns
df = df.append(df2)
df = df.reset_index(drop=True)
df['品名'] = df['品名'].apply(lambda x:x.strip())
display(df[df['品名']=='MA'])
產品集合 = set(df['品名'].values)

,RIGID,分機,公司代號,公司事業部門,品名
531,NaN,NaN,2P,南亞塑四部化學品部,MA
1480,NaN,NaN,2P,南亞塑四部化學品部,MA


In [4]:
df[df['品名'] == 'SAN']

,RIGID,分機,公司代號,公司事業部門,品名
1047,NaN,NaN,4A,塑膠事業部,SAN
1414,NaN,NaN,4A,塑膠事業部,SAN


In [5]:
df[df['品名'] == 'INA']

,RIGID,分機,公司代號,公司事業部門,品名
465,郭慶怡,6246,27,南亞化一部,INA
1925,郭慶怡,6246,27,南亞化一部,INA


In [6]:
品名2部門 = dict(zip(df['品名'],df['公司事業部門']))
品名2代號 = dict(zip(df['品名'],df['公司代號']))

In [7]:
val_df = pd.read_csv('../data/preprocess_for_SQUAD_產品.csv',index_col=0)[['string_X_train','Y_label','EXPNO']].dropna(axis=0)
print(val_df.shape)
val_df

(4492, 3)


,string_X_train,Y_label,EXPNO
3,MASS PVC RESIN B57 QUANTITY 175 MT AT 1300 USD...,PVC RESIN B57,11
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27
16,COMMODITY TAISOX 7470MQUANTITY100MTUNIT PRICEU...,TAISOX,18
18,COUNTRY OF ORIGIN TAIWANPRICE TERMS CIF BU S...,SAN,XU
19,COUNTRY OF ORIGIN TAIWANPRICE TERMS CIF BU S...,SAN,XU
...,...,...,...
20638,MALEIC ANHYDRIDE QUANTITY 6000 MT AT USD 1460...,MALEIC ANHYDRIDE,2P
20652,DESCRIPTION OF GOODS ...,EPOXY RESIN,2R
20657,PLASTIC RESIN CIF WUZHOU CH INA1 1 X 20 FCL CO...,PLASTIC RESIN,1P
20709,ABS AG15AAHQUANTITY 28800MT UNIT PRICE USD238...,INA,4A


In [8]:
train_df = val_df

# 如果品名是單詞的話 前後加個空白

In [9]:
for i in df.index:
    name = df.loc[i,'品名'].strip()
    df.loc[i,'品名'] = f' {name} '
    assert df.loc[i,'品名'][0] == ' '
    assert df.loc[i,'品名'][-1] == ' '

In [10]:
display(df[df['品名']=='MA'])
display(df[df['品名']==' MA '])

,RIGID,分機,公司代號,公司事業部門,品名


,RIGID,分機,公司代號,公司事業部門,品名
531,NaN,NaN,2P,南亞塑四部化學品部,MA
1480,NaN,NaN,2P,南亞塑四部化學品部,MA


In [11]:
產品集合 = set(df['品名'].values)

# find_fail_sample and drop fail_sample

In [12]:
def find_fail_sample(df):
    fails = []
    for i in df.index:
        context = df.loc[i,'string_X_train']
        answer = df.loc[i,'Y_label']
        if str(answer) not in str(context):
            fails.append(i)
    return fails
train_fails = find_fail_sample(train_df)
val_fails = find_fail_sample(val_df)
print(train_fails,val_fails)
display(val_df.loc[val_fails])
print(val_df.shape)
val_df = val_df.drop(val_fails,axis=0)
print(val_df.shape)

[] []


,string_X_train,Y_label,EXPNO


(4492, 3)
(4492, 3)


In [13]:
str('abc')[str('abc').find('b')+1:]

'c'

In [14]:
def Collection_method(df,產品集合):
    labels = {}
    for i in tqdm(df.index):
        products = []
        for p in 產品集合:
            x = df.loc[i,'string_X_train']
            if p in x:
                if ' ' in p: # 非單詞直接append
                    products.append(p)
                if ' ' not in p: # 單詞要判斷一下
                    if ('INA' in x) and ('CHINA' not in x) :# 去除CHINA中找到INA這種情況
                        products.append(p)
        labels[i] = products
    predict = pd.DataFrame(index=labels.keys(),columns=['predict'])
    predict['predict'] = labels.values()
    return predict
predict = Collection_method(val_df,產品集合)
result = val_df.join(predict)
result['class'] = 'rule'

In [15]:
result

,string_X_train,Y_label,EXPNO,predict,class
3,MASS PVC RESIN B57 QUANTITY 175 MT AT 1300 USD...,PVC RESIN B57,11,"[ RESIN , PVC RESIN ]",rule
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,[],rule
16,COMMODITY TAISOX 7470MQUANTITY100MTUNIT PRICEU...,TAISOX,18,[ TAISOX ],rule
18,COUNTRY OF ORIGIN TAIWANPRICE TERMS CIF BU S...,SAN,XU,[ SAN ],rule
19,COUNTRY OF ORIGIN TAIWANPRICE TERMS CIF BU S...,SAN,XU,[ SAN ],rule
...,...,...,...,...,...
20638,MALEIC ANHYDRIDE QUANTITY 6000 MT AT USD 1460...,MALEIC ANHYDRIDE,2P,[ EVA ],rule
20652,DESCRIPTION OF GOODS ...,EPOXY RESIN,2R,"[ RESIN , NPEL , EPOXY RESIN , MA ]",rule
20657,PLASTIC RESIN CIF WUZHOU CH INA1 1 X 20 FCL CO...,PLASTIC RESIN,1P,"[ RESIN , PP ]",rule
20709,ABS AG15AAHQUANTITY 28800MT UNIT PRICE USD238...,INA,4A,[],rule


In [16]:
not_find = []
for j,i in enumerate(result['predict'].values):
    if len(i) == 0:
        not_find.append(j)
len(not_find)

388

In [17]:
not_find_df = result.iloc[not_find]
not_find_df

,string_X_train,Y_label,EXPNO,predict,class
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,[],rule
33,CIF BU SANORIGIN TAIWAN GLASS FIBER YARNECG75...,SAN,XU,[],rule
34,CIF BU SANORIGIN TAIWAN GLASS FIBER YARNECG75...,SAN,XU,[],rule
128,PVC SUS PENSION S65 112 MT AT USD 133000 PER ...,EA,11,[],rule
173,TERMS OF PRICE CIF BU SANHS CODE 7019190000D...,SAN,XU,[],rule
...,...,...,...,...,...
20633,PRECIPITATED CALCIUM CARBONATE FP100AQUANTITY...,PRECIPITATED CALCIUM CARBONATE,14,[],rule
20634,PRECIPITATED CALCIUM CARBONATE FP100AQUANTITY...,PRECIPITATED CALCIUM CARBONATE,14,[],rule
20635,PRECIPITATED CALCIUM CARBONATE FP100AQUANTITY...,PRECIPITATED CALCIUM CARBONATE,14,[],rule
20636,PRECIPITATED CALCIUM CARBONATE FP100AQUANTITY...,PRECIPITATED CALCIUM CARBONATE,14,[],rule


In [18]:
bert_predict = model_predict(nlp,not_find_df)
bert_predict

,predict:
4,PHTHALIC ANHYDRIDE
33,GLASS FIBER
34,GLASS FIBER
128,PVC SUS PENSION S65
173,SANHS CODE 7019190000DESCRIPTION GLASS FIBER
...,...
20633,CALCIUM CARBONATE
20634,CALCIUM CARBONATE
20635,CALCIUM CARBONATE
20636,CALCIUM CARBONATE


In [19]:
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [20]:
result.iloc[not_find,3] = [ [str(i)] for i in bert_predict['predict:'].values]
idx = result.iloc[not_find].index
result.loc[idx,'class'] = 'bert'
result.loc[idx]

,string_X_train,Y_label,EXPNO,predict,class
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,[PHTHALIC ANHYDRIDE],bert
33,CIF BU SANORIGIN TAIWAN GLASS FIBER YARNECG75...,SAN,XU,[GLASS FIBER],bert
34,CIF BU SANORIGIN TAIWAN GLASS FIBER YARNECG75...,SAN,XU,[GLASS FIBER],bert
128,PVC SUS PENSION S65 112 MT AT USD 133000 PER ...,EA,11,[PVC SUS PENSION S65],bert
173,TERMS OF PRICE CIF BU SANHS CODE 7019190000D...,SAN,XU,[SANHS CODE 7019190000DESCRIPTION GLASS FIBER],bert
...,...,...,...,...,...
20633,PRECIPITATED CALCIUM CARBONATE FP100AQUANTITY...,PRECIPITATED CALCIUM CARBONATE,14,[CALCIUM CARBONATE],bert
20634,PRECIPITATED CALCIUM CARBONATE FP100AQUANTITY...,PRECIPITATED CALCIUM CARBONATE,14,[CALCIUM CARBONATE],bert
20635,PRECIPITATED CALCIUM CARBONATE FP100AQUANTITY...,PRECIPITATED CALCIUM CARBONATE,14,[CALCIUM CARBONATE],bert
20636,PRECIPITATED CALCIUM CARBONATE FP100AQUANTITY...,PRECIPITATED CALCIUM CARBONATE,14,[CALCIUM CARBONATE],bert


In [21]:
result.loc[idx]

,string_X_train,Y_label,EXPNO,predict,class
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,[PHTHALIC ANHYDRIDE],bert
33,CIF BU SANORIGIN TAIWAN GLASS FIBER YARNECG75...,SAN,XU,[GLASS FIBER],bert
34,CIF BU SANORIGIN TAIWAN GLASS FIBER YARNECG75...,SAN,XU,[GLASS FIBER],bert
128,PVC SUS PENSION S65 112 MT AT USD 133000 PER ...,EA,11,[PVC SUS PENSION S65],bert
173,TERMS OF PRICE CIF BU SANHS CODE 7019190000D...,SAN,XU,[SANHS CODE 7019190000DESCRIPTION GLASS FIBER],bert
...,...,...,...,...,...
20633,PRECIPITATED CALCIUM CARBONATE FP100AQUANTITY...,PRECIPITATED CALCIUM CARBONATE,14,[CALCIUM CARBONATE],bert
20634,PRECIPITATED CALCIUM CARBONATE FP100AQUANTITY...,PRECIPITATED CALCIUM CARBONATE,14,[CALCIUM CARBONATE],bert
20635,PRECIPITATED CALCIUM CARBONATE FP100AQUANTITY...,PRECIPITATED CALCIUM CARBONATE,14,[CALCIUM CARBONATE],bert
20636,PRECIPITATED CALCIUM CARBONATE FP100AQUANTITY...,PRECIPITATED CALCIUM CARBONATE,14,[CALCIUM CARBONATE],bert


In [22]:
lst = []
for i in result.iloc[not_find].index:
    if result.loc[i,'Y_label'] != result.loc[i,'predict']:
        lst.append(i)

In [23]:
'PVC SUSPENSION S65' in 產品集合

False

In [24]:
' EA ' in 產品集合

True

In [25]:
'SAN' in 產品集合 

False

In [26]:
get_jaccard_sim('MASS PVC RESIN B-57','PVC RESIN B-57')

0.75

In [27]:
def get_acc(df,t=0.75):
    correct = []
    correct_label = []
    for i in df.index:
        jacs = []
        for j in df.loc[i,'predict']:
            jacs.append(get_jaccard_sim(df.loc[i,'Y_label'],j))
        if max(jacs) >= t:
            correct.append('yes')
        else:
            correct.append('no')
    result = pd.DataFrame({'correct':correct})
    return result['correct'].value_counts()['yes']/len(result)

In [28]:
def get_jac(df):
    all_jacs = []
    for i in df.index:
        jacs = []
        for j in df.loc[i,'predict']:
            jacs.append(get_jaccard_sim(df.loc[i,'Y_label'],j))
        all_jacs.append(max(jacs))
    return np.sum(all_jacs)/len(all_jacs)

# 表現

In [29]:
get_acc(result,t=1),get_acc(result,t=0.75),get_jac(result)

(0.6431433659839715, 0.6560552092609083, 0.7422751558325912)

In [30]:
部門_lst = []
for p_lst in tqdm(result['predict'].values):
    p = max(p_lst,key=len)
    jac_dict = {}
    for i in 品名2代號.keys():
        jac_dict[i] = get_jaccard_sim(i,p)
    部門_lst.append(品名2代號[max(jac_dict, key=jac_dict.get)])

In [31]:
result['預測部門代號'] = 部門_lst

In [32]:
result['predict'] = [substringSieve(i) for i in result['predict']]
result

,string_X_train,Y_label,EXPNO,predict,class,預測部門代號
3,MASS PVC RESIN B57 QUANTITY 175 MT AT 1300 USD...,PVC RESIN B57,11,[ PVC RESIN ],rule,11
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,[PHTHALIC ANHYDRIDE],bert,27
16,COMMODITY TAISOX 7470MQUANTITY100MTUNIT PRICEU...,TAISOX,18,[ TAISOX ],rule,18
18,COUNTRY OF ORIGIN TAIWANPRICE TERMS CIF BU S...,SAN,XU,[ SAN ],rule,4A
19,COUNTRY OF ORIGIN TAIWANPRICE TERMS CIF BU S...,SAN,XU,[ SAN ],rule,4A
...,...,...,...,...,...,...
20638,MALEIC ANHYDRIDE QUANTITY 6000 MT AT USD 1460...,MALEIC ANHYDRIDE,2P,[ EVA ],rule,18
20652,DESCRIPTION OF GOODS ...,EPOXY RESIN,2R,"[ NPEL , EPOXY RESIN , MA ]",rule,2R
20657,PLASTIC RESIN CIF WUZHOU CH INA1 1 X 20 FCL CO...,PLASTIC RESIN,1P,"[ RESIN , PP ]",rule,4A
20709,ABS AG15AAHQUANTITY 28800MT UNIT PRICE USD238...,INA,4A,[INA],bert,27


In [33]:
substringSieve([ 'TAISOX' , 'LLDPE TAISOX'])

['LLDPE TAISOX']

In [34]:
result['EXPNO'] = [ str(i).strip() for i in result['EXPNO']]
result['預測部門代號'] = [ str(i).strip() for i in result['預測部門代號']]

In [35]:
result[result['EXPNO']==result['預測部門代號']]

,string_X_train,Y_label,EXPNO,predict,class,預測部門代號
3,MASS PVC RESIN B57 QUANTITY 175 MT AT 1300 USD...,PVC RESIN B57,11,[ PVC RESIN ],rule,11
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,[PHTHALIC ANHYDRIDE],bert,27
16,COMMODITY TAISOX 7470MQUANTITY100MTUNIT PRICEU...,TAISOX,18,[ TAISOX ],rule,18
31,PRODUCT TETRAHYDROFURAN 998 PCT MINQUANTITY...,TETRAHYDROFURAN,2P,"[ EA , TETRAHYDROFURAN , EVA , MA ]",rule,2P
41,875 MTS OF PVC RESIN B57 AND 360 MTS OF PVC ...,PVC RESIN S65D,11,"[ EVA , PVC RESIN ]",rule,11
...,...,...,...,...,...,...
20634,PRECIPITATED CALCIUM CARBONATE FP100AQUANTITY...,PRECIPITATED CALCIUM CARBONATE,14,[CALCIUM CARBONATE],bert,14
20635,PRECIPITATED CALCIUM CARBONATE FP100AQUANTITY...,PRECIPITATED CALCIUM CARBONATE,14,[CALCIUM CARBONATE],bert,14
20636,PRECIPITATED CALCIUM CARBONATE FP100AQUANTITY...,PRECIPITATED CALCIUM CARBONATE,14,[CALCIUM CARBONATE],bert,14
20652,DESCRIPTION OF GOODS ...,EPOXY RESIN,2R,"[ NPEL , EPOXY RESIN , MA ]",rule,2R


In [36]:
result[result['EXPNO']!=result['預測部門代號']]

,string_X_train,Y_label,EXPNO,predict,class,預測部門代號
18,COUNTRY OF ORIGIN TAIWANPRICE TERMS CIF BU S...,SAN,XU,[ SAN ],rule,4A
19,COUNTRY OF ORIGIN TAIWANPRICE TERMS CIF BU S...,SAN,XU,[ SAN ],rule,4A
22,300000 BBLS 10PCT OF GASOIL 10 PPMUNIT PRICE ...,GASOIL,61,"[ PA , GASOIL , MA ]",rule,60
23,300000 BBLS 10PCT OF GASOIL 10 PPMUNIT PRICE ...,GASOIL,61,"[ PA , GASOIL , MA ]",rule,60
24,300000 BBLS 10PCT OF GASOIL 10 PPMUNIT PRICE ...,GASOIL,61,"[ PA , GASOIL , MA ]",rule,60
...,...,...,...,...,...,...
20587,LLD PE GRADE 38400QUANTITY 300 MTUNIT PRICE US...,INA,18,[ PE ],rule,21
20588,LLD PE GRADE 38400QUANTITY 300 MTUNIT PRICE US...,INA,18,[ PE ],rule,21
20638,MALEIC ANHYDRIDE QUANTITY 6000 MT AT USD 1460...,MALEIC ANHYDRIDE,2P,[ EVA ],rule,18
20657,PLASTIC RESIN CIF WUZHOU CH INA1 1 X 20 FCL CO...,PLASTIC RESIN,1P,"[ RESIN , PP ]",rule,4A


In [37]:
a = len(result[result['EXPNO']==result['預測部門代號']])
b = len(result[result['EXPNO']!=result['預測部門代號']])
print(f'正確:{a} 錯誤:{b} 正確率:{a/(a+b)}')

正確:2745 錯誤:1747 正確率:0.6110863757791629


In [38]:
品名2代號['GASOIL'] # EXPNO是61 寶典是60

'60'

In [39]:
len(result[result['EXPNO']==result['預測部門代號']])/len(result)

0.6110863757791629

In [40]:
result.to_csv('submit_product_0906.csv')

In [41]:
result.head(30)

,string_X_train,Y_label,EXPNO,predict,class,預測部門代號
3,MASS PVC RESIN B57 QUANTITY 175 MT AT 1300 USD...,PVC RESIN B57,11,[ PVC RESIN ],rule,11
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,[PHTHALIC ANHYDRIDE],bert,27
16,COMMODITY TAISOX 7470MQUANTITY100MTUNIT PRICEU...,TAISOX,18,[ TAISOX ],rule,18
18,COUNTRY OF ORIGIN TAIWANPRICE TERMS CIF BU S...,SAN,XU,[ SAN ],rule,4A
19,COUNTRY OF ORIGIN TAIWANPRICE TERMS CIF BU S...,SAN,XU,[ SAN ],rule,4A
22,300000 BBLS 10PCT OF GASOIL 10 PPMUNIT PRICE ...,GASOIL,61,"[ PA , GASOIL , MA ]",rule,60
23,300000 BBLS 10PCT OF GASOIL 10 PPMUNIT PRICE ...,GASOIL,61,"[ PA , GASOIL , MA ]",rule,60
24,300000 BBLS 10PCT OF GASOIL 10 PPMUNIT PRICE ...,GASOIL,61,"[ PA , GASOIL , MA ]",rule,60
30,300000 BBLS 10PCT OF GASOIL 10 PPMUNIT PRICE ...,GASOIL,61,"[ PA , GASOIL , MA ]",rule,60
31,PRODUCT TETRAHYDROFURAN 998 PCT MINQUANTITY...,TETRAHYDROFURAN,2P,"[ EA , TETRAHYDROFURAN , EVA , MA ]",rule,2P
